In [ ]:
# triage_agent.py
#
# 1. Classifies a support ticket (simple vs complex)
# 2. Generates an automated reply for simple tickets
# 3. Escalates complex tickets by e-mailing the help-desk queue
#
# Uses LangGraph for explicit workflow control
#
# Dependencies for this agent demo:
#   LLM API: openai==1.51.0 (or anthropic==0.34.1 for Claude)
#   Frameworks: langchain==0.3.0, langgraph==0.2.14
#   Data validation: pydantic==2.8.2
#   Environment: python-dotenv==1.0.1
#
#   pip install openai==1.51.0 langchain==0.3.0 langgraph==0.2.14 pydantic==2.8.2 python-dotenv==1.0.1
#
#   Or create requirements.txt with exact pins:
#   openai==1.51.0
#   langchain==0.3.0
#   langgraph==0.2.14
#   pydantic==2.8.2
#   python-dotenv==1.0.1
#   Then: pip install -r requirements.txt
#


#   export OPENAI_API_KEY="sk-..."
#   python triage_agent.py

from typing import Literal, TypedDict, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

load_dotenv()

# ────────────────────────────────────────────────────────────────────────────────
# Pydantic models for structured output
# ────────────────────────────────────────────────────────────────────────────────
class TicketClassification(BaseModel):
    """Classification result for a support ticket."""
    classification: Literal["SIMPLE", "COMPLEX"] = Field(description="Ticket complexity classification")
    reasoning: str = Field(description="Brief explanation of the classification decision")
    priority: Literal["LOW", "MEDIUM", "HIGH"] = Field(description="Ticket priority level")

class CustomerResponse(BaseModel):
    """Structured customer response."""
    subject: str = Field(description="Email subject line")
    body: str = Field(description="Email body content in British English")
    next_steps: List[str] = Field(description="List of next steps for the customer")

# ────────────────────────────────────────────────────────────────────────────────
# State definition
# ────────────────────────────────────────────────────────────────────────────────
class TriageState(TypedDict):
    ticket: str
    customer_email: str
    classification: str
    response: str

# ────────────────────────────────────────────────────────────────────────────────
# Tool functions with schemas
# ────────────────────────────────────────────────────────────────────────────────
@tool
def escalate_ticket(ticket: str, customer_email: str, priority: str, reasoning: str) -> str:
    """Escalate a complex ticket to human support.

    Args:
        ticket: The original support ticket text
        customer_email: Customer's email address
        priority: Ticket priority (LOW, MEDIUM, HIGH)
        reasoning: Why this ticket needs human attention
    """
    # In production, this would create a ticket in your support system
    print(f"TICKET ESCALATED")
    print(f"Customer: {customer_email}")
    print(f"Priority: {priority}")
    print(f"Reasoning: {reasoning}")
    print(f"Ticket: {ticket[:100]}...")
    return f"Ticket escalated successfully with {priority} priority"

@tool
def send_customer_response(customer_email: str, subject: str, body: str, next_steps: List[str]) -> str:
    """Send an automated response to the customer.

    Args:
        customer_email: Customer's email address
        subject: Email subject line
        body: Email body content
        next_steps: List of next steps for the customer
    """
    # In production, this would send via your email service
    print(f"📧 CUSTOMER RESPONSE SENT")
    print(f"To: {customer_email}")
    print(f"Subject: {subject}")
    print(f"Body:\n{body}")
    print(f"Next steps: {', '.join(next_steps)}")
    return f"Response sent successfully to {customer_email}"

# ────────────────────────────────────────────────────────────────────────────────
# LLM setup
# ────────────────────────────────────────────────────────────────────────────────
llm = ChatOpenAI(model="gpt-4.1", temperature=0.1)

# ────────────────────────────────────────────────────────────────────────────────
# Node functions
# ────────────────────────────────────────────────────────────────────────────────
def classify_ticket(state: TriageState) -> TriageState:
    """Classify ticket using structured output."""
    classification_prompt = ChatPromptTemplate.from_template("""
You are an expert in customer support triage.

Classify this ticket with:
- classification: SIMPLE (can be answered with template) or COMPLEX (needs human intervention)
- reasoning: Brief explanation of your decision
- priority: LOW, MEDIUM, or HIGH based on urgency and impact

Ticket: {ticket}
""")

    # Use structured output with Pydantic model
    structured_llm = llm.with_structured_output(TicketClassification)
    chain = classification_prompt | structured_llm
    result = chain.invoke({"ticket": state["ticket"]})

    return {**state, "classification": result.classification}

def generate_response(state: TriageState) -> TriageState:
    """Generate structured response for simple tickets."""
    response_prompt = ChatPromptTemplate.from_template("""
Create a customer support response for this ticket:

- subject: Appropriate email subject line
- body: Polite reply in British English, less than 150 words, signed as 'Support Team'
- next_steps: List of 2-3 actionable next steps for the customer

Be concise, empathetic and resolve their issue.

Ticket: {ticket}
""")

    # Use structured output
    structured_llm = llm.with_structured_output(CustomerResponse)
    chain = response_prompt | structured_llm
    result = chain.invoke({"ticket": state["ticket"]})

    # Use the send_customer_response tool
    send_result = send_customer_response.invoke({
        "customer_email": state["customer_email"],
        "subject": result.subject,
        "body": result.body,
        "next_steps": result.next_steps
    })

    return {**state, "response": send_result}

def escalate_ticket_node(state: TriageState) -> TriageState:
    """Escalate complex tickets using the escalate_ticket tool."""
    # First, get classification details for escalation
    classification_prompt = ChatPromptTemplate.from_template("""
Analyze this support ticket for escalation:

Ticket: {ticket}
""")

    structured_llm = llm.with_structured_output(TicketClassification)
    chain = classification_prompt | structured_llm
    result = chain.invoke({"ticket": state["ticket"]})

    # Use the escalate_ticket tool
    escalation_result = escalate_ticket.invoke({
        "ticket": state["ticket"],
        "customer_email": state["customer_email"],
        "priority": result.priority,
        "reasoning": result.reasoning
    })

    return {**state, "response": escalation_result}

# ────────────────────────────────────────────────────────────────────────────────
# Routing logic
# ────────────────────────────────────────────────────────────────────────────────
def route_ticket(state: TriageState) -> Literal["generate_response", "escalate_ticket_node"]:
    """Route based on classification."""
    if state["classification"] == "SIMPLE":
        return "generate_response"
    else:
        return "escalate_ticket_node"

# ────────────────────────────────────────────────────────────────────────────────
# Build the workflow graph
# ────────────────────────────────────────────────────────────────────────────────
def create_triage_workflow():
    """Create the triage workflow graph."""
    workflow = StateGraph(TriageState)

    # Add nodes
    workflow.add_node("classify_ticket", classify_ticket)
    workflow.add_node("generate_response", generate_response)
    workflow.add_node("escalate_ticket_node", escalate_ticket_node)

    # Define the flow
    workflow.set_entry_point("classify_ticket")
    workflow.add_conditional_edges(
        "classify_ticket",
        route_ticket,
        {
            "generate_response": "generate_response",
            "escalate_ticket_node": "escalate_ticket_node"
        }
    )
    workflow.add_edge("generate_response", END)
    workflow.add_edge("escalate_ticket_node", END)

    return workflow.compile()

# ────────────────────────────────────────────────────────────────────────────────
# Main function
# ────────────────────────────────────────────────────────────────────────────────
def triage_ticket(ticket: str, customer_email: str = "") -> None:
    """Process a support ticket through the triage workflow."""
    app = create_triage_workflow()

    result = app.invoke({
        "ticket": ticket,
        "customer_email": customer_email,
        "classification": "",
        "response": ""
    })

    if result["classification"] == "SIMPLE":
        print("\n--- AUTOMATED REPLY ---\n", result["response"], "\n")
    else:
        print("Ticket escalated to help-desk queue.")


In [ ]:
sample_ticket = """
Hi, my parcel arrived damaged and the replacement you promised
last month never showed up. I need it before Friday or I'll cancel.
"""
triage_ticket(sample_ticket, "alice@example.com")

In [ ]:
sample_ticket = """
Hi, when do parcels usually get delivered?
"""
triage_ticket(sample_ticket, "bob@example.com")

In [ ]:
sample_ticket = """
I accidentally sent a £1,000,000 ring in a parcel without any insurance, is this okay?
"""
triage_ticket(sample_ticket, "rich@example.com")